In [ ]:
# -*- coding: utf-8 -*-
"""
---------------------------------------------------------------------------------------------------
 How to Set Up Direct Access the LP DAAC Data Pool with Python
 The following Python code will configure a netrc profile that will allow users to download data
 from an Earthdata Login enabled server. See README for additional information
---------------------------------------------------------------------------------------------------
 Author: Cole Krehbiel
 Last Updated: 11/20/2018
-------------------------------------------------------------------------------
"""
# Load necessary packages into Python
from netrc import netrc
from subprocess import Popen
from getpass import getpass
import os

# -----------------------------------AUTHENTICATION CONFIGURATION-------------------------------- #
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)

In [ ]:
import ast
import os
import csv
from datetime import datetime
import fiona
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import json
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from pathlib import Path
import pickle
import pyproj
from pyproj import Proj
from shapely.ops import transform
import xarray as xr
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')
# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
directory='.../Illisoy/Illinois_shp/county_shape_files/ToJSON'

CSlist = os.listdir(directory)

print(CSlist)

In [ ]:
allDates = []

for filename in CSlist:
    
    dirname0 = '.../Illisoy/RSMeta/2018'
    
    monthnumber = os.listdir(dirname0)
    
    suffix0 = '.txt'
    
    for number in monthnumber:
        
        with open(os.path.join(dirname0, number, Path(os.path.join(directory, filename)).stem + suffix0)) as f:
    
            hls_items = ast.literal_eval(f.read())    
        
        # GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
        gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
        gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
        gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
        gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
    
        for j, h in enumerate(hls_items):

            vi_band_links = []

# Define which HLS product is being accessed
            if h['assets']['browse']['href'].split('/')[4] == 'HLSS30.020':
                vi_bands = ['B12', 'B11', 'B8A', 'B04', 'B03', 'B02', 'Fmask'] # Bands for S30
            else:
                vi_bands = ['B07', 'B06', 'B05', 'B04', 'B03', 'B02', 'Fmask'] # Bands for L30

# Subset the assets in the item down to only the desired bands
            for a in h['assets']: 
                if any(b == a for b in vi_bands):
                   vi_band_links.append(h['assets'][a]['href'])

            try:
                for e in vi_band_links:
                    if e.rsplit('.', 2)[-2] == vi_bands[0]:      # SWIR2 index
                        swir2 = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[1]:    # SWIR1 index
                        swir1 = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[2]:    # NIR index
                        nir = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[3]:    # red index
                        red = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[4]:    # green index
                        green = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[5]:    # blue index
                        blue = rio.open(e)
                    elif e.rsplit('.', 2)[-2] == vi_bands[6]:    # Fmask index
                        fmask = rio.open(e)
                    
            except:
                print(f"{h['assets']['browse']['href'].split('/')[-1].rsplit('.', 1)[0]} ({h['id']}) is not recognized as a supported file format")
                continue

            geo_CRS = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)  # Source coordinate system of the ROI
            utm = pyproj.Proj(nir.crs)                                                  # Destination coordinate system
            project = pyproj.Transformer.from_proj(geo_CRS, utm)                        # Set up the transformation
        
            field = gp.read_file(os.path.join(directory, filename))
            fieldShape = field['geometry'][0] # Define the geometry as a shapely polygon
            fsUTM = transform(project.transform, fieldShape)                            # Apply reprojection
        
            try:

                nir_array, nir_transform = rio.mask.mask(nir, [fsUTM], crop=True)  # Extract the data for the ROI and clip to that bbox
                swir2_array, _ = rio.mask.mask(swir2,[fsUTM],crop=True)
                swir1_array, _ = rio.mask.mask(swir1,[fsUTM],crop=True)
                green_array, _ = rio.mask.mask(green,[fsUTM],crop=True)
                red_array, _ = rio.mask.mask(red,[fsUTM],crop=True)
                blue_array, _ = rio.mask.mask(blue,[fsUTM],crop=True)
            
            except:
                print(f"{h['assets']['browse']['href'].split('/')[-1].rsplit('.', 1)[0]} ({h['id']} input shapes do not overlap raster, move to the next file.")
                continue
            
            swir2_scaled = swir2_array[0] * 0.0001
            swir1_scaled = swir1_array[0] * 0.0001
            nir_scaled = nir_array[0] * 0.0001
            red_scaled = red_array[0] * 0.0001
            green_scaled = green_array[0] * 0.0001
            blue_scaled = blue_array[0] * 0.0001

            swir2_scaled[swir2_array[0]==swir2.nodata] = np.nan 
            swir1_scaled[swir1_array[0]==swir1.nodata] = np.nan 
            nir_scaled[nir_array[0]==nir.nodata] = np.nan 
            red_scaled[red_array[0]==red.nodata] = np.nan 
            green_scaled[green_array[0]==green.nodata] = np.nan 
            blue_scaled[blue_array[0]==blue.nodata] = np.nan 
        
            dir_name = '.../Illisoy/CDL-Illisoy/2018'
            
            suffix = '.tif'
    
            cropmask = gdal.Open(os.path.join(dir_name, Path(os.path.join(directory, filename)).stem + suffix))

            cmarray = np.array(cropmask.GetRasterBand(1).ReadAsArray())
        
            cmarray = np.resize(cmarray, np.shape(nir_scaled))

            swir2_scaled = np.ma.masked_array(swir2_scaled, mask = cmarray != 5)
            swir1_scaled = np.ma.masked_array(swir1_scaled, mask = cmarray != 5)
            nir_scaled = np.ma.masked_array(nir_scaled, mask = cmarray != 5)
            red_scaled = np.ma.masked_array(red_scaled, mask = cmarray != 5)
            green_scaled = np.ma.masked_array(green_scaled, mask = cmarray != 5)
            blue_scaled = np.ma.masked_array(blue_scaled, mask = cmarray != 5)   
        

            def ndvi(red, nir):
                return (nir - red) / (nir + red)

            def tvi(red, nir):
                return pow((nir - red) / (nir + red) + 0.5, 0.5) * 100

            def evi(red, blue, nir):
                return 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0) 

            def satvi(swir1, swir2, red):
                return 2 * ((swir1 - red) / (swir1 + red + 1)) - 0.5 * swir2

            def savi(nir, red):
                return 1.5 * (nir - red) /(nir - red + 0.5)

            def msi(swir1, nir):
                return swir1 / nir
    
            def gndvi(nir, green):
                return (nir - green) / (nir + green)
  
            def grvi(green, red):
                return (green - red) / (green + red)
    
            def lswi(nir, swir1):
                return (nir - swir1) / (nir + swir1)
    
            def tsavi(nir, red):
                return 1.2 * (nir - 1.2 * red - 0.04) / (0.04 * nir + red - 0.04 * 1.2 + 0.08 * (1 + 1.2 * 1.2))
  
            def msavi(nir, red):
                return (1 + 1 - 2 * 1.06 * ((nir - red) / (nir + red)) * (nir - 1.06 * red)) * (nir - red) / (nir + red + 1 - 2 * 1.06 * ((nir - red) / (nir + red)) * (nir - 1.06 * red))
  
            def msavi2(nir, red):
                return (2 * nir + 1 - pow(pow((2 * nir + 1), 2) - 8 * (nir - red), 0.5) ) / 2

            def wdvi(nir, red):
                return nir - 1.06 * red
      
            def bi(red, green):
                return pow((pow(red, 2) + pow(green, 2)), 0.5) / 2
  
            def bi2(red, green, nir):
                return pow(pow(red, 2) + pow(green, 2) + pow(nir, 2), 0.5) / 3
    
            def ri(red, green):
                return pow(red, 2) / pow(green, 3)
  
            def ci(red, green):
                return (red - green) / (red + green)
  
            def v(nir, red):
                return nir / red

# Generate VI array

            ndvi_scaled = ndvi(red_scaled, nir_scaled)

            tvi_scaled = tvi(red_scaled, nir_scaled)

            evi_scaled = evi(red_scaled, blue_scaled, nir_scaled) 

            satvi_scaled = satvi(swir1_scaled, swir2_scaled, red_scaled)

            savi_scaled = savi(nir_scaled, red_scaled)

            msi_scaled = msi(swir1_scaled, nir_scaled)
   
            gndvi_scaled = gndvi(nir_scaled, green_scaled)

            grvi_scaled = grvi(green_scaled, red_scaled)

            lswi_scaled = lswi(nir_scaled, swir1_scaled)
 
            tsavi_scaled = tsavi(nir_scaled, red_scaled)
  
            msavi_scaled = msavi(nir_scaled, red_scaled)

            msavi2_scaled = msavi2(nir_scaled, red_scaled)

            wdvi_scaled = wdvi(nir_scaled, red_scaled)

            bi_scaled = bi(red_scaled, green_scaled)
  
            bi2_scaled = bi2(red_scaled, green_scaled, nir_scaled)
    
            ri_scaled = ri(red_scaled, green_scaled)
  
            ci_scaled = ci(red_scaled, green_scaled)
  
            v_scaled = v(nir_scaled, red_scaled)

            fmask_array, _ = rio.mask.mask(fmask, [fsUTM], crop=True)  # Load in the Quality data

            bitword_order = (1, 1, 1, 1, 1, 1, 2)  # set the number of bits per bitword
            num_bitwords = len(bitword_order)      # Define the number of bitwords based on your input above
            total_bits = sum(bitword_order)        # Should be 8, 16, or 32 depending on datatype

            qVals = list(np.unique(fmask_array))  # Create a list of unique values that need to be converted to binary and decoded
            all_bits = list()
    
            goodQuality = []
    
            for v in qVals:
                all_bits = []
                bits = total_bits
                i = 0
    
    # Convert to binary based on the values and # of bits defined above:
                bit_val = format(v, 'b').zfill(bits)
                print('\n' + str(v) + ' = ' + str(bit_val))
                all_bits.append(str(v) + ' = ' + str(bit_val))

    # Go through & split out the values for each bit word based on input above:
                for b in bitword_order:
                    prev_bit = bits
                    bits = bits - b
                    i = i + 1
                    if i == 1:
                        bitword = bit_val[bits:]
                        print(' Bit Word ' + str(i) + ': ' + str(bitword))
                        all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
                    elif i == num_bitwords:
                        bitword = bit_val[:prev_bit]
                        print(' Bit Word ' + str(i) + ': ' + str(bitword))
                        all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
                    else:
                        bitword = bit_val[bits:prev_bit]
                        print(' Bit Word ' + str(i) + ': ' + str(bitword))
                        all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
            
    # 2, 4, 5, 6 are the bits used. All 4 should = 0 if no clouds, cloud shadows were present, and pixel is not snow/ice/water
                if int(all_bits[2].split(': ')[-1]) + int(all_bits[4].split(': ')[-1]) + \
                int(all_bits[5].split(': ')[-1]) + int(all_bits[6].split(': ')[-1]) == 0:
                    goodQuality.append(v)
        
## rewrite it as a loop

            swir2_band = np.ma.MaskedArray(swir2_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            swir2_band = np.ma.filled(swir2_band, np.nan)                                                 # Set masked data to nan

            swir1_band = np.ma.MaskedArray(swir1_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            swir1_band = np.ma.filled(swir1_band, np.nan)                                                 # Set masked data to nan

            nir_band = np.ma.MaskedArray(nir_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            nir_band = np.ma.filled(nir_band, np.nan)                                                 # Set masked data to nan

            red_band = np.ma.MaskedArray(red_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            red_band = np.ma.filled(red_band, np.nan)                                                 # Set masked data to nan

            green_band = np.ma.MaskedArray(green_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            green_band = np.ma.filled(green_band, np.nan)                                                 # Set masked data to nan

            blue_band = np.ma.MaskedArray(blue_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            blue_band = np.ma.filled(blue_band, np.nan)                                                 # Set masked data to nan

            ndvi_band = np.ma.MaskedArray(ndvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            ndvi_band = np.ma.filled(ndvi_band, np.nan)                                                 # Set masked data to nan

            tvi_band = np.ma.MaskedArray(tvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            tvi_band = np.ma.filled(tvi_band, np.nan)                                                 

            evi_band = np.ma.MaskedArray(evi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            evi_band = np.ma.filled(evi_band, np.nan)                                                 # Set masked data to nan

            satvi_band = np.ma.MaskedArray(satvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            satvi_band = np.ma.filled(satvi_band, np.nan)                                                 # Set masked data to nan

            savi_band = np.ma.MaskedArray(savi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            savi_band = np.ma.filled(savi_band, np.nan)                                                 # Set masked data to nan

            msi_band = np.ma.MaskedArray(msi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            msi_band = np.ma.filled(msi_band, np.nan)                                                 # Set masked data to nan

            gndvi_band = np.ma.MaskedArray(gndvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            gndvi_band = np.ma.filled(gndvi_band, np.nan)                                                 # Set masked data to nan

            grvi_band = np.ma.MaskedArray(grvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            grvi_band = np.ma.filled(grvi_band, np.nan)                                                 # Set masked data to nan

            lswi_band = np.ma.MaskedArray(lswi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            lswi_band = np.ma.filled(lswi_band, np.nan)                                                 # Set masked data to nan

            tsavi_band = np.ma.MaskedArray(tsavi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            tsavi_band = np.ma.filled(tsavi_band, np.nan)                                                 # Set masked data to nan

            msavi_band = np.ma.MaskedArray(msavi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            msavi_band = np.ma.filled(msavi_band, np.nan)                                                 # Set masked data to nan

            msavi2_band = np.ma.MaskedArray(msavi2_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            msavi2_band = np.ma.filled(msavi2_band, np.nan)                                                 # Set masked data to nan

            wdvi_band = np.ma.MaskedArray(wdvi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            wdvi_band = np.ma.filled(wdvi_band, np.nan)                                                 # Set masked data to nan

            bi_band = np.ma.MaskedArray(bi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            bi_band = np.ma.filled(bi_band, np.nan)                                                 # Set masked data to nan

            bi2_band = np.ma.MaskedArray(bi2_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            bi2_band = np.ma.filled(bi2_band, np.nan)                                                 # Set masked data to nan

            ri_band = np.ma.MaskedArray(ri_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            ri_band = np.ma.filled(ri_band, np.nan)                                                 # Set masked data to nan

            ci_band = np.ma.MaskedArray(ci_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            ci_band = np.ma.filled(ci_band, np.nan)                                                 # Set masked data to nan

            v_band = np.ma.MaskedArray(v_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
            v_band = np.ma.filled(v_band, np.nan)                                                 # Set masked data to nan

        # Remove any observations that are entirely fill value
            if np.nansum(evi_band) == 0.0:
                print(f"File: {h['assets']['browse']['href'].split('/')[-1].rsplit('.', 1)[0]} ({h['id']}) was entirely fill values and will not be exported.")
                continue
        
            SWIR2 = np.nanmean(swir2_band)
            SWIR1 = np.nanmean(swir1_band)
            NIR = np.nanmean(nir_band)
            RED = np.nanmean(red_band)
            GREEN = np.nanmean(green_band)
            BLUE = np.nanmean(blue_band)
            NDVI = np.nanmean(ndvi_band)
            TVI = np.nanmean(tvi_band)
            EVI = np.nanmean(evi_band)
            SATVI = np.nanmean(satvi_band)
            SAVI = np.nanmean(savi_band)
            MSI = np.nanmean(msi_band)
            GNDVI = np.nanmean(gndvi_band)
            GRVI = np.nanmean(grvi_band)
            LSWI = np.nanmean(lswi_band)
            TSAVI = np.nanmean(tsavi_band)
            MSAVI = np.nanmean(msavi_band)
            MSAVI2 = np.nanmean(msavi2_band)
            WDVI = np.nanmean(wdvi_band)
            BI = np.nanmean(bi_band)
            BI2 = np.nanmean(bi2_band)
            RI = np.nanmean(ri_band)
            CI = np.nanmean(ci_band)
            V = np.nanmean(v_band)
       
            viDate = h['properties']['datetime'].split('T')[0]  # Set the observation date to a variable
            SL = h['assets']['browse']['href'].split('/')[4]
            
 # dictionary and add Date
            VIvec = {'County': Path(os.path.join(directory, filename)).stem, 'Monthnumber': number[-1:], 'Date': viDate, 'SL': SL,
                     'SWIR2': SWIR2, 'SWIR1': SWIR1, 'NIR': NIR,
                     'RED': RED, 'GREEN': GREEN, 'BLUE': BLUE,
                     'NDVI': NDVI, 'TVI': TVI, 'EVI': EVI,
                     'SATVI': SATVI, 'SAVI': SAVI, 'MSI': MSI,
                     'GNDVI': GNDVI, 'GRVI': GRVI, 'LSWI': LSWI,
                     'TSAVI': TSAVI, 'MSAVI': MSAVI, 'MSAVI2': MSAVI2,
                     'WDVI': WDVI, 'BI': BI, 'BI2': BI2,
                     'RI': RI, 'CI': CI, 'V': V}   

            allDates.append(VIvec)
    
print(allDates)

In [ ]:
print(allDates)
VIvec = {'County': Path(os.path.join(directory, filename)).stem, 'Monthnumber': number[-1:], 'Date': viDate, 'SL': SL,
         'SWIR2': SWIR2, 'SWIR1': SWIR1, 'NIR': NIR,
         'RED': RED, 'GREEN': GREEN, 'BLUE': BLUE,
         'NDVI': NDVI, 'TVI': TVI, 'EVI': EVI,
         'SATVI': SATVI, 'SAVI': SAVI, 'MSI': MSI,
         'GNDVI': GNDVI, 'GRVI': GRVI, 'LSWI': LSWI,
         'TSAVI': TSAVI, 'MSAVI': MSAVI, 'MSAVI2': MSAVI2,
         'WDVI': WDVI, 'BI': BI, 'BI2': BI2,
         'RI': RI, 'CI': CI, 'V': V}

# save as csv.
with open('2018VI.csv', 'w') as f:  
    w = csv.DictWriter(f, VIvec.keys())
    w.writeheader()
    w.writerows(allDates)